In [4]:
pip install python-dotenv

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import tweepy
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import time
from datetime import datetime

load_dotenv()

True

In [8]:
consumer_key = os.environ["TWITTER_API_KEY"]
consumer_secret = os.environ["TWITTER_API_KEY_SECRET"]
access_token = os.environ["TWITTER_ACCESS_TOKEN"]
access_token_secret = os.environ["TWITTER_ACCESS_TOKEN_SECRET"]

In [9]:
auth = tweepy.OAuth1UserHandler(
    consumer_key, 
    consumer_secret,
    access_token,
    access_token_secret
    )

api = tweepy.API(auth)
api


In [11]:
me = api.verify_credentials()


In [12]:
search_country = 'India'

In [13]:
def scrape_tweets(search_words_list, date_since, date_until, num_tweets, num_runs, search_country):
    
    tweets_df = pd.DataFrame(columns = ['user_id', 'tweet_id', 'username', 'location', 
                                        'following', 'followers', 'twt_created_at', 'total_tweets', 
                                        'retweet_count', 'text', 'hashtags', 'mentions'])
    
    program_start = time.time()
    
    places = api.search_geo(query=search_country, granularity="country")
    place_id = places[0].id
    count_num = 0
    count_tweet = 0
    for search_words in search_words_list:
        
        for i in range(0, num_runs):
            count_num+=1
            no_tweet = 0
            
            start_run = time.time()
            
            tweets = tweepy.Cursor(api.search_tweets, q=search_words + ' AND ' + ("place:%s" % place_id), lang='en', tweet_mode='extended').items(num_tweets)
            print("count_num = ", count_num)
            
        
            
            tweet_list = [tweet for tweet in tweets]
            print(len(tweet_list))
            # print(tweet_list)
            for tweet in tweet_list:
                count_tweet+=1
                if tweet.created_at.date() <= datetime.strptime(date_until, '%Y-%m-%d').date():
                    
                    user_id = tweet.user.id
                    tweet_id = tweet.id
                    username = tweet.user.screen_name
                    location = tweet.user.location
                    following = tweet.user.friends_count
                    followers = tweet.user.followers_count
                    twt_created_at = tweet.created_at
                    total_tweets = tweet.user.statuses_count
                    retweet_count = tweet.retweet_count
                    hashtags = tweet.entities['hashtags']
                    mentions = tweet.entities['user_mentions']
                
                    try:    
                        text = tweet.retweeted_status.full_text
                    except AttributeError:
                        text = tweet.full_text
                    # print(text)
                    ith_tweet = [user_id, tweet_id, username, location, 
                                following, followers, twt_created_at ,total_tweets, 
                                retweet_count, text, hashtags, mentions]
                    
                    tweets_df.loc[len(tweets_df)] = ith_tweet
            
                    no_tweet += 1
            print("count_tweet = ", count_tweet)
            time.sleep(920)

            end_run = time.time()
            duration_run = round((end_run-start_run)/60, 2)
            
            print('Number of tweets scraped for run {} is {}'.format(i+1, no_tweet))
            print('Time take for {} run to complete is {} mins'.format(i+1, duration_run))
    
    
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
    
    path = os.getcwd()
    filename = path + '/data/' + to_csv_timestamp + '_tweets.csv'
    tweets_df.to_csv(filename, index=False)
    
    program_end = time.time()
    print('Scraping is completed.')    
    print('Total time taken to scrape is {} minutes.'.format(round(program_end-program_start)/60, 2))
    
    return tweets_df
    


In [14]:
# search_queries = ["hygienic OR medical OR lack OR facilities OR beds OR medicines OR hygienic OR hygiene OR #emergency OR #complaints OR #medicalnegligence OR #patientdeath OR @MoHFW_INDIA OR @PMOIndia OR @ArvindKejriwal OR @mansukhmandviya OR @ANI OR @PTI_News OR @narendramodi"] 

In [46]:
search_words_list = ["(#safdarjunghospital understaffed) OR (aiims complaint) OR (@ArvindKejriwal look into matter) OR (#rmlhospital neglegience) OR #aiims OR #safdarjunghospital OR #rmlhospital OR (@MCD_Delhi neglegience) OR (@SBM_MCD complaint) OR (@MCD_Delhi protest)",
                    "(@MCD_Delhi look into matter) OR (@MCD_Delhi critical condition) OR (@CPCB_OFFICIAL look into matter) OR (@CPCB_OFFICIAL mcd complaint) OR (@SatyendarJain shame hospital) OR (@ArvindKejriwal shame hospital) OR (@MCD_Delhi no action) OR (@SatyendarJain look into matter) OR (@narendramodi look into matter)"]
date_since = '2020-01-01'
date_until = '2022-11-08'
num_tweets = 2000
num_runs = 3
# time.sleep(920)
scrape_tweets(search_words_list, date_since, date_until, num_tweets, num_runs, search_country)

count_num =  1
25
count_tweet =  25
Number of tweets scraped for run 1 is 18
Time take for 1 run to complete is 15.4 mins
count_num =  2
25
count_tweet =  50
Number of tweets scraped for run 2 is 18
Time take for 2 run to complete is 15.38 mins
count_num =  3
25
count_tweet =  75
Number of tweets scraped for run 3 is 18
Time take for 3 run to complete is 15.39 mins
count_num =  4
24
count_tweet =  99
Number of tweets scraped for run 1 is 21
Time take for 1 run to complete is 15.41 mins
count_num =  5
24
count_tweet =  123
Number of tweets scraped for run 2 is 21
Time take for 2 run to complete is 15.4 mins
count_num =  6
24
count_tweet =  147
Number of tweets scraped for run 3 is 21
Time take for 3 run to complete is 15.39 mins
Scraping is completed.
Total time taken to scrape is 92.38333333333334 minutes.


,user_id,tweet_id,username,location,following,followers,twt_created_at,total_tweets,retweet_count,text,hashtags,mentions
0,1383680115641511938,1590008526344458240,ganga_vihar,,20,20,2022-11-08 15:49:08+00:00,1125,0,@DelhiComplaint @MCD_Delhi No acknowledgement ...,[],"[{'screen_name': 'DelhiComplaint', 'name': 'De..."
1,130842599,1589906021199523842,eraviteja,PAN INDIA,720,218,2022-11-08 09:01:49+00:00,1589,0,"Rather than Quantity, Quality is alao required...","[{'text': 'purvanchal', 'indices': [77, 88]}, ...",[]
2,1383680115641511938,1589310922535100416,ganga_vihar,,20,20,2022-11-06 17:37:06+00:00,1125,0,@dcshahsouth @GyaneshBharti1 @MCD_Delhi @SBM_M...,[],"[{'screen_name': 'dcshahsouth', 'name': 'Deput..."
3,1396360829067423751,1588521464214716416,drvinay_aiims,"New Delhi, India",241,1712,2022-11-04 13:20:05+00:00,916,6,🔈✳️\nHeights of Good governance\n\n#aiims in u...,"[{'text': 'aiims', 'indices': [32, 38]}, {'tex...","[{'screen_name': 'ANI', 'name': 'ANI', 'id': 3..."
4,1396360829067423751,1588482216753258498,drvinay_aiims,"New Delhi, India",241,1712,2022-11-04 10:44:07+00:00,916,8,🛺🛺🛺🛺🛺\n50 more E-Vehicles with #free service w...,"[{'text': 'free', 'indices': [30, 35]}, {'text...","[{'screen_name': 'ANI', 'name': 'ANI', 'id': 3..."
...,...,...,...,...,...,...,...,...,...,...,...,...
112,1069549705414279173,1587816126972841985,rwabblk,,115,26,2022-11-02 14:37:19+00:00,201,4,@DPCC_pollution @LtGovDelhi @URJADelhi @MCD_De...,[],"[{'screen_name': 'DPCC_pollution', 'name': 'De..."
113,1235474608205303808,1587758361851797504,SunilGaur69,"Rohini, New Delhi",273,179,2022-11-02 10:47:47+00:00,3874,2,@rajeev_baisoya @Navin00879697 @DCCNZMCD @MCD_...,[],"[{'screen_name': 'rajeev_baisoya', 'name': 'Ra..."
114,352858184,1587445323869278208,bimand1,NEW DELHI,169,80,2022-11-01 14:03:53+00:00,4259,0,@narendramodi Sir 🙏 small children and elderly...,"[{'text': 'Action', 'indices': [149, 156]}, {'...","[{'screen_name': 'narendramodi', 'name': 'Nare..."
115,725347577957257218,1587374087223275520,rksharma183,"New Delhi, India",82,5,2022-11-01 09:20:49+00:00,78,0,@EESL_India @MCD_Delhi I don't have complaint ...,[],"[{'screen_name': 'EESL_India', 'name': 'EESL I..."


In [ ]:
search_words_list = ["'@ArvindKejriwal #aiims' OR #patientdeath OR #unionhealthministry OR #hospital OR @MoHFW_INDIA OR @PMOIndia OR @ArvindKejriwal OR '@ArvindKejriwal #neglegience' OR @ANI OR @PTI_News OR '@ArvindKejriwal #safdarjunghospital'",
                    "beds OR vaccines OR streetlights OR #mcdcomplaint OR '@ArvindKejriwal #hospital' OR '@PMOIndia #safdarjunghospital' OR @MCD_Delhi OR '@PMOIndia #rmlhospital' OR '@ArvindKejriwal #complaint' OR @DCkpzon OR @HMOIndia OR @AshwaniKumar_92 OR @narendramodi OR @PMOIndia OR @DelhiComplaint OR @SBM_MCD OR medical+protest OR #MedTwitter"]
date_since = '2020-01-01'